In [ ]:
#Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_percentage_error,mean_absolute_error,root_mean_squared_error
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.svm import SVR
import shap



scale_cols = [
        'AVG temperature', 'AVG rain', 'AVG fr_rain', 'AVG snow', 'AVG ice', 'AVG snow_depth',
        'AVG accumulated', 'AVG probability', 'AVG rate', 'AVG clouds', 'Physical capacity village level',
        'Physical capacity village level for hotel', 'Physical capacity village level for comfort',
        'Physical capacity village level for premium', 'Physical capacity village level for VIP', 
        'Rented villa nights before booking date on village level',
        'Rented villa nights before booking date on village level for hotel',
        'Rented villa nights before booking date on village level for comfort',
        'Rented villa nights before booking date on village level for premium',
        'Rented villa nights before booking date on village level for VIP',
        'Cottage rent per person', 'Lagged Cancellation Amount','Lagged cancellation insurance Amount','Lagged travel insurance Amount',
        'MIN temperature', 'MAX temperature', 'MIN rain', 'MAX rain',
        'MIN fr_rain', 'MAX fr_rain', 'MIN snow', 'MAX snow', 'MIN ice', 'MAX ice', 'MIN snow_depth', 
        'MAX snow_depth', 'MIN accumulated', 'MAX accumulated', 'MIN probability', 'MAX probability',
        'MIN rate', 'MAX rate'
    ]
weather_cols=['AVG temperature', 'AVG rain', 'AVG fr_rain', 'AVG snow', 'AVG ice', 'AVG snow_depth',
        'AVG accumulated', 'AVG probability', 'AVG rate', 'AVG clouds','MIN temperature', 'MAX temperature', 'MIN rain', 'MAX rain',
        'MIN fr_rain', 'MAX fr_rain', 'MIN snow', 'MAX snow', 'MIN ice', 'MAX ice', 'MIN snow_depth', 
        'MAX snow_depth', 'MIN accumulated', 'MAX accumulated', 'MIN probability', 'MAX probability',
        'MIN rate', 'MAX rate']
capacity_cols=[ 'Lagged Cancellation Amount','Lagged cancellation insurance Amount','Lagged travel insurance Amount',
               'Physical capacity village level','Physical capacity village level for hotel', 'Physical capacity village level for comfort',
            'Physical capacity village level for premium', 'Physical capacity village level for VIP', 
            'Rented villa nights before booking date on village level',
            'Rented villa nights before booking date on village level for hotel',
            'Rented villa nights before booking date on village level for comfort',
            'Rented villa nights before booking date on village level for premium',
            'Rented villa nights before booking date on village level for VIP',
            'Cottage rent per person']
cat_cols=['Holiday name', 'Leadtime','Booking day of week', 'Day of week visit',
        'Booking month', 'Month visit', 'Booking day date', 'Day visit date']
lag_cols=['Rented villa nights before booking date on village level',
       'Rented villa nights before booking date on village level for hotel',
       'Rented villa nights before booking date on village level for comfort',
       'Rented villa nights before booking date on village level for premium',
       'Rented villa nights before booking date on village level for VIP','Lagged Cancellation Amount', 'Lagged cancellation insurance Amount',
       'Lagged travel insurance Amount','AVG temperature', 'AVG rain', 'AVG fr_rain', 'AVG snow', 'AVG ice', 'AVG snow_depth',
        'AVG accumulated', 'AVG probability', 'AVG rate', 'AVG clouds','MIN temperature', 'MAX temperature', 'MIN rain', 'MAX rain',
        'MIN fr_rain', 'MAX fr_rain', 'MIN snow', 'MAX snow', 'MIN ice', 'MAX ice', 'MIN snow_depth', 
        'MAX snow_depth', 'MIN accumulated', 'MAX accumulated', 'MIN probability', 'MAX probability',
        'MIN rate', 'MAX rate']



# Splitting data in train, test and validation
def train_test_validation(df):
    split_date = '2023-09-30'
    split_date2 = '2022-10-01'
    split_date3 = '2022-09-30'
    train = df.loc[:split_date3]
    validation = df.loc[split_date2:split_date]
    test = df.loc[split_date:]

    return train,test,validation

#compares predicted and true values based on pandas describe
#extreme argument indicates what deviation of prediction and true value is classified as extreme
def comparison(y_test, y_pred, lead, compare, algo,X_test,extreme=0.2932751092707807):
    
    start_date= '2023-10-01'
    end_date= '2024-09-02'

    # Append new data if 'compare' already exists
    new_data = pd.DataFrame({
        'true': y_test,
        'predicted': y_pred,
        'Leadtime': lead,
        'Model': algo,
        'difference': y_test - y_pred  ,
        'negative': y_pred < 0,
        'extreme': (y_test - y_pred > extreme) | (y_test - y_pred < -extreme),
        'extreme threshold':extreme,
        'Date visit':X_test.index
    })
    
    compare = pd.concat([compare, new_data], ignore_index=True)
    
    
    return compare
#Fills missing values based on last observed value closest to date in specified limit, optionally can also fill with maximum value for that date
#column1 is the column with missing values
#column is the date column that the imputation is based on
#days is the upper limit that is used for looking up past values
def fill_values(df, column1, column2, days, maxi=False):
    # Convert the date column to datetime
    df[column2] = pd.to_datetime(df[column2], format='%d-%m-%Y', errors='raise')

    # Get indices of NaN values in column1
    na_indices = df[df[column1].isna()].index

    # Iterate through the indices with NaN values
    for index in na_indices:
        filled = False  # Track if we filled the NaN
        current_date = df.loc[index, column2]

        # Get rows for the same date
        same_date_rows = df[df[column2] == current_date]

        if maxi:
            # If maxi=True, fill with the max value from that day
            max_value = same_date_rows[column1].max()
            if not pd.isna(max_value):
                df.loc[index, column1] = max_value
                filled = True
        else:
            # Fill with the first non-NaN value from that day
            for _, row in same_date_rows.iterrows():
                if not pd.isna(row[column1]):
                    df.loc[index, column1] = row[column1]
                    filled = True
                    break

        # If not filled from the same date, check previous days
        if not filled and days > 0:
            for day in range(1, days + 1):
                prev_date = current_date - pd.Timedelta(days=day)
                prev_rows = df[df[column2] == prev_date]

                if maxi:
                    # Fill with max value from the previous date
                    max_value = prev_rows[column1].max()
                    if not pd.isna(max_value):
                        df.loc[index, column1] = max_value
                        filled = True
                        break
                else:
                    # Fill with first non-NaN value from previous dates
                    for _, row in prev_rows.iterrows():
                        if not pd.isna(row[column1]):
                            df.loc[index, column1] = row[column1]
                            filled = True
                            break
                if filled:
                    break
        
        # If no value was filled after checking the same and previous days
        if not filled:
            print(f"NaN found at index {index} for date {df.loc[index, column2]} could not be filled with any values.")

    return df
#Turns date visit, forecast dt iso, slice dt iso, and booking date into pandas date formats
def date_format():
    df['Date visit']=pd.to_datetime(df['Date visit'], format='%d-%m-%Y', errors='raise')
    df['Booking date'] = pd.to_datetime(df["Booking date"], format='%d-%m-%Y', errors='raise')
    df['forecast dt iso'] = pd.to_datetime(df["forecast dt iso"], format='%d-%m-%Y', errors='raise')
    df['slice dt iso'] = pd.to_datetime(df["slice dt iso"], format='%d-%m-%Y', errors='raise')
#Creates leadtime, day of week, month features for booking date and date visit
def feature_engineer():
    df['Leadtime']=(df['Date visit'] - df['Booking date']).dt.days.astype(int)
    df['Booking day of week']=df['Booking date'].dt.dayofweek
    df['Day of week visit']=df['Date visit'].dt.dayofweek
    df['Booking month']=df['Booking date'].dt.month
    df['Month visit']=df['Date visit'].dt.month 
    df['Booking day date']=df['Booking date'].dt.day
    df['Day visit date']=df['Date visit'].dt.day
#Filters df for leadtimes between 0 and up to 16 days
def leadfilter ():
    #Filtering for leadtime matching the weather forecast: Between 0 and 16 days
    df=df[df['Leadtime']>=0]
    df=df[df['Leadtime']<=16]
#Aggregated insurance counts are filled with 0, as nan means 0 for those columns
def impute():
    
    df[['Cancellation amount','Cancellation insurance', 'Travel insurance']] = df[['Cancellation amount','Cancellation insurance', 'Travel insurance']].fillna(0)     
# replaces commas with points and turns columns numeric
def make_numeric():
#Replace commas with points and turn object columns to numeric
    cols = [
        'MIN temperature', 'MAX temperature', 'AVG temperature',
        'MIN rain', 'MAX rain', 'AVG rain',
        'MIN fr_rain', 'MAX fr_rain', 'AVG fr_rain',
        'MIN snow', 'MAX snow', 'AVG snow',
        'MIN ice', 'MAX ice', 'AVG ice',
        'MIN snow_depth', 'MAX snow_depth', 'AVG snow_depth',
        'MIN accumulated', 'MAX accumulated', 'AVG accumulated',
        'MIN probability', 'MAX probability', 'AVG probability',
        'MIN rate', 'MAX rate', 'AVG rate',
        'AVG clouds','Cottage rent per person','Cottage rent VIP per person',
        'Cottage rent comfort per person','Cottage rent premium per person'

    ]



    def replace_commas(x):
        if isinstance(x, str):
            return x.replace(',', '.')
        return x

    # Apply replacement to relevant columns
    df[cols] = df[cols].apply(lambda col: col.map(replace_commas))
    df[cols] = df[cols].apply(pd.to_numeric, errors='raise')

# Function to propagate lagged values for a given test DataFrame and leadtime
def propagate_lag_values(test, leadtime, lag_cols=lag_cols):
    leadtime_values = test[test['Leadtime'] == leadtime][lag_cols]
    
    # Ensure dynamic alignment of values row-by-row
    for col in lag_cols:
        test[col] = test.index.map(leadtime_values[col].to_dict())
    test = test[test['Leadtime'] != leadtime].copy()
    return test


filepath='C:\\Users\\lsugg\\OneDrive - Pierre & Vacances - Centerparcs Group\\Desktop\\Sixth try.csv'
data=pd.read_csv(filepath,delimiter=';',low_memory=False)
df=pd.DataFrame(data)
#Some observations that are based on calculations in tableau have the '#DIV/0' adn get replaced with na
df.replace('#DIV/0', np.nan, inplace=True)

print(df.info())


date_format()
feature_engineer()
#df['Occupancy'] = df['Rented villa nights in date visit'] / df['Physical capacity village level']
# Filling missing holiday names based on holiday names for the same date visit
df = fill_values(df, 'Holiday name', 'Date visit', 0)
#Turning Holiday name categorical
print(np.unique(df['Holiday name']))
holiday_list = ['Autumn H', 'Christ_NY H', 'Core Summer H', 'Easter May H', 'Spring H', 'Summer H']
df['Holiday name'] = np.where(df['Holiday name'].isin(holiday_list), 1, 0)


#No forecast for this time
df = df[~(df['Date visit'] >= '2024-09-17')]
#Park was closed from 19.11.2018 until 06.12.2018
df = df[~((df['Date visit'] >= '2018-11-19') & (df['Date visit'] <= '2018-12-06'))]
ddf=df.copy()
df = df.sort_values(by=['Date visit', 'Booking date'], ascending=[True, True]).set_index('Date visit') 
impute()
make_numeric()
df.info()

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [ ]:

#Dropping observations without weather data
df=df.dropna(subset='forecast dt iso')

# Split overview
split_date = '2023-09-30' #end of training-validation data
split_date2 = '2022-10-01' # start of validation data
split_date3 = '2022-09-30' # end of training data
#5% na values for renting price
# Filling NA with training median to avoid data leakage
train_median = df['Cottage rent per person'].loc[:split_date3].median()
df['Cottage rent per person'] = df['Cottage rent per person'].fillna(train_median)
#Dropping temp columns
df=df.drop(['Rented villa nights',
       'Rented villa nights hotel', 'Rented villa nights comfort',
       'Rented villa nights premium', 'Rented villa nights VIP','Booking date', 'forecast dt iso', 'slice dt iso'],axis=1)
#Dropping all columns with missing values
df=df.dropna(axis=1)
#columns used for preprocessing in tableau
df=df.drop(['Rented villa nights in date visit for VIP1',
       'Rented villa nights in date visit for VIP',
       'Rented villa nights in date visit for premium1',
       'Rented villa nights in date visit for premium',
       'Rented villa nights in date visit for comfort1',
       'Rented villa nights in date visit for comfort',
       'Rented villa nights in date visit for hotel',
       'Rented villa nights in date visit1',
       'Rented villa nights in date visit'],axis=1)
def create_lag():

       # Create a lagged version of 'Cancellation amount' by shifting within groups
       df['Lagged Cancellation Amount'] = df.groupby(df.index)['Cancellation amount'].shift(1)
       # Compute the running sum of the lagged values
       df['Lagged Cancellation Amount'] = df.groupby(df.index)['Lagged Cancellation Amount'].cumsum().fillna(0)

       # Create a lagged version of 'cancellation insurance amount' by shifting within groups
       df['Lagged cancellation insurance Amount'] = df.groupby(df.index)['Cancellation insurance'].shift(1)
       # Compute the running sum of the lagged values
       df['Lagged cancellation insurance Amount'] = df.groupby(df.index)['Lagged cancellation insurance Amount'].cumsum().fillna(0)

       # Create a lagged version of 'travel insurance amount' by shifting within groups
       df['Lagged travel insurance Amount'] = df.groupby(df.index)['Travel insurance'].shift(1)
       # Compute the running sum of the lagged values
       df['Lagged travel insurance Amount'] = df.groupby(df.index)['Lagged travel insurance Amount'].cumsum().fillna(0)
       for col in weather_cols:
              df[col]=df.groupby(df.index)[col].shift(1)
create_lag()        

df['Occupancy']=df[df['Leadtime']==0]['Rented villa nights before booking date on village level']/df[df['Leadtime']==0]['Physical capacity village level']
print(df[['Leadtime', 'Cancellation amount','Lagged Cancellation Amount','Cancellation insurance','Lagged cancellation insurance Amount','Travel insurance','Lagged travel insurance Amount']].loc['2023-08-04'].head(17))
#Dropping features after lagged features have been created
df=df.drop(['Cancellation amount',	'Cancellation insurance',	'Travel insurance'],axis=1)
#Filter rows where 'Occupancy' is na
na_rows = df[df['Occupancy'].isna()]
#As missing rows are close to the end of provided data, there is no 0 lead time, thus the 17 rows are dropped
# Display the rows
print(na_rows[['Leadtime','Occupancy']].head(17))
df=df.dropna()
df.columns


In [ ]:
df_corr = df.corr()
df_corr=df_corr.drop(['Physical capacity village level',
       'Physical capacity village level for hotel',
       'Physical capacity village level for comfort',
       'Physical capacity village level for premium',
       'Physical capacity village level for VIP'],axis=1)
df_corr=df_corr.drop(['Physical capacity village level',
       'Physical capacity village level for hotel',
       'Physical capacity village level for comfort',
       'Physical capacity village level for premium',
       'Physical capacity village level for VIP'],axis=0)
useful_df = pd.DataFrame()

for row, col in zip(df_corr.index, df_corr.columns[::-1]):
    if row in weather_cols and col in weather_cols:
        continue
    elif row in capacity_cols and col in capacity_cols:
        continue
    elif row == col:
        continue
    else:
        # Extract the correlation value
        new_row = pd.DataFrame([df_corr.loc[row, col]], columns=[f'{row}-{col}'])
        useful_df = pd.concat([useful_df, new_row], axis=1)

#transposing the final df to have rows as correlations
useful_df = useful_df.transpose()
useful_df['Correlation']=useful_df[0].copy()
useful_df=useful_df.drop([0],axis=1)
useful_df.sort_values(by=['Correlation'])
useful_df=useful_df.dropna()
useful_df
plt.figure(figsize=(6,10))
plt.barh(useful_df.index, useful_df.iloc[:, 0])  
plt.xticks(rotation=90)  

plt.ylabel('Correlation Pairs')  
plt.xlabel('Correlation Value')  
plt.title('Correlation Values Between Features')  

plt.show()


In [ ]:
df_corr=df.corr()
df_corr=df_corr.sort_values(by=['Occupancy'])
plt.figure(figsize=(20,8))
bars = plt.bar(df_corr.index, df_corr['Occupancy'])

# Add annotations (index names) on top of each bar

plt.axhline(0.2,color='r')
plt.axhline(-0.2,color='r')
# Set labels and title
plt.xlabel('Variables')
plt.ylabel('Correlation Values')
plt.title('Correlation of "Occupancy"')

# Display the plot
plt.xticks(rotation=90)  # Rotate x-axis labels if needed
plt.show()


In [3]:
horizon14 = df[df['Leadtime'] <= 15].copy()
horizon7 = df[df['Leadtime'] <= 8].copy()
horizon5 = df[df['Leadtime'] <= 6].copy()
horizon1 = df[df['Leadtime'] <= 2].copy()
horizon14 = propagate_lag_values(horizon14, 15).dropna()
horizon7 = propagate_lag_values(horizon7, 8).dropna()
horizon5 = propagate_lag_values(horizon5, 6).dropna()
horizon1 = propagate_lag_values(horizon1, 2).dropna()
frames=[horizon14,horizon7,horizon5,horizon1]

In [ ]:
df.to_csv('Prepped_data_occupancy.csv')

In [ ]:
df=pd.DataFrame(pd.read_csv('C:\\Users\\lsugg\OneDrive - Pierre & Vacances - Centerparcs Group\\Desktop\\Thesis\\Prepped_data_occupancy.csv'))


In [ ]:


# Define lead colors for visualization
lead_colors = {1: 'blue'}

# Plotting target variable
plt.figure(figsize=(20, 8))

# Loop through unique lead times and plot each series
for lead in [1]:
    temp_df = df[df['Leadtime'] == lead]
    print(temp_df['Occupancy'].describe())
    # Group by date and calculate mean of 'Rented villa nights to come' per date
    mean_series = temp_df.groupby(temp_df.index)['Occupancy'].mean()
    plt.plot(temp_df['Occupancy'], color=lead_colors.get(lead, 'black'))

# Add legend and adjust layout
plt.axhline(temp_df['Occupancy'].mean(),label='Mean')
plt.axhline(temp_df['Occupancy'].median(),label='Median',color='r')
plt.legend()
plt.xlabel("Date visit")
plt.ylabel("Occupancy")
plt.title("Occupancy distribution")
plt.tight_layout()  # Adjust layout to make room for the legend
plt.show()


In [ ]:
# Splitting data in train, test and validation
split_date = '2023-09-30' #end of training-validation data
split_date2 = '2022-10-01' # start of validation data
split_date3 = '2022-09-30' # end of training data


def scree_separate(train):
    leads = [1, 5, 7, 14]


    scaler = RobustScaler()
    frame = train.copy()
    lead=frame['Leadtime'].max()
    #Scaling the training data for the specified columns
    frame_scaled = scaler.fit_transform(frame[scale_cols])
    frame_scaled = pd.DataFrame(frame_scaled, columns=scale_cols, index=frame.index)
    
    #Applying PCA to weather and capacity features
    pca_weather = PCA(n_components=10)
    pca_capacity = PCA(n_components=10)

    train_weather = pd.DataFrame(
        pca_weather.fit_transform(frame_scaled[weather_cols]),
        columns=[f'PC_weather_{i+1}' for i in range(10)],
        index=frame.index
    )
    
    train_capacity = pd.DataFrame(
        pca_capacity.fit_transform(frame_scaled[capacity_cols]),
        columns=[f'PC_capacity_{i+1}' for i in range(10)],
        index=frame.index
    )
    
    #Combining PCA results
    pca_separate = pd.concat([train_weather, train_capacity], axis=1)
    
    #Plotting the scree plots
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    #Scree plot for weather features
    axes[0].plot(range(1, 11), pca_weather.explained_variance_ratio_, marker='o', label='Weather')
    axes[0].set_title(f'Scree Plot for Weather PCA (Leadtime {lead})')
    axes[0].set_xlabel('Principal Component')
    axes[0].set_ylabel('Explained Variance Ratio')
    axes[0].grid()
    axes[0].legend()

    #Scree plot for capacity features
    axes[1].plot(range(1, 11), pca_capacity.explained_variance_ratio_, marker='o', label='Capacity')
    axes[1].set_title(f'Scree Plot for Capacity PCA (Leadtime {lead})')
    axes[1].set_xlabel('Principal Component')
    axes[1].set_ylabel('Explained Variance Ratio')
    axes[1].grid()
    axes[1].legend()

    plt.tight_layout()
    plt.show()



def scree_blended (train):
    leads=[1,5,7,1]
    
    scaler=RobustScaler()
    frame = train.copy()
    lead=frame['Leadtime'].max()
    
    train_scaled = scaler.fit_transform(frame[scale_cols])

    #PCA
    pca_all = PCA(n_components=10)

    #Fitting PCA on the entire scaled dataset
    pca_all.fit(train_scaled)

    #Creating a df for the explained variance ratio
    explained_variance_ratio = pd.DataFrame(pca_all.explained_variance_ratio_, 
                                            columns=['Explained Variance Ratio'])

    #Plotting the scree plot 
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, 11), explained_variance_ratio.values, marker='o')
    plt.title(f'Scree Plot for PCA on All Scaled Columns with Leadtime {lead}')
    plt.xlabel('Principal Component')
    plt.ylabel('Explained Variance Ratio')
    plt.xticks(range(1, 11))
    plt.grid()
    plt.show()

for frame in frames: 
    train = frame.loc[:split_date3]

    validation = frame.loc[split_date2:split_date]
    test = frame.loc[split_date:]
    scree_separate(train=train)

In [ ]:
for frame in frames: 
    train = frame.loc[:split_date3]

    validation = frame.loc[split_date2:split_date]
    test = frame.loc[split_date:]
    scree_blended(train)

In [ ]:
#naive model uses prior year to predict test year
#
def naive_model(split_date = '2023-10-01'):
        #point forecast
    ddf=df.copy()
    ddf=ddf.drop(index='2024-02-29')
    lead16=ddf[ddf['Leadtime']<=14]
    lead7=ddf[ddf['Leadtime']<=7]
    lead5=ddf[ddf['Leadtime']<=5]
    lead1=ddf[ddf['Leadtime']<=1]
    maes={}
    compare = pd.DataFrame(columns=['true','predicted','Leadtime','difference','negative','Date visit','extreme'])

    frames=[lead16, lead7, lead5, lead1]
    for frame in frames:
        # Perform the time-based split
        #scale(frame,'Occupancy')
        lead=frame['Leadtime'].max()
        
        split_date = '2023-10-01'
        #remove leap year date
        
        test = frame.loc[split_date:]
        test=test[['Occupancy','Leadtime']]
        end='2024-09-01'
        test=test.loc[:end]
        split_date2 = '2022-10-01'
        train_naive=frame.loc[split_date2:]
        #Because that is where the test data stops
        split_date3='2023-09-01'
        train_naive=train_naive.loc[:split_date3]
        train_naive=train_naive[['Occupancy','Leadtime']]
        #train_naive now contains only the dates 1 year prior to the test data

        naive_pred=train_naive['Occupancy'].reset_index(drop=True)
        actual=test['Occupancy'].reset_index(drop=True)


        # Calculate metrics
        mape= np.round(mean_absolute_percentage_error(actual, naive_pred), 3)
        mae = np.round(mean_absolute_error(actual, naive_pred), 3)
        rmse = np.round(root_mean_squared_error(actual, naive_pred), 3)
        
        
        compare=comparison(actual,naive_pred,lead,compare,"Naive",test)
    
        maes.update({lead.astype(str):mae})
    print(f'Naive forecast from previous year')
    print('MAE: ', mae)
    print('RMSE: ', rmse)
    
    print('MAPE:',mape)
    print('')
    print('')
    
    return compare
residuals_naive=naive_model()


In [ ]:
residuals_naive

In [5]:
capacity_only=['Occupancy','Holiday name',
 'Leadtime',
 'Booking day of week',
 'Day of week visit',
 'Booking month',
 'Month visit',
 'Booking day date',
 'Day visit date','Lagged Cancellation Amount',
 'Lagged cancellation insurance Amount',
 'Lagged travel insurance Amount',
 'Physical capacity village level',
 'Physical capacity village level for hotel',
 'Physical capacity village level for comfort',
 'Physical capacity village level for premium',
 'Physical capacity village level for VIP',
 'Rented villa nights before booking date on village level',
 'Rented villa nights before booking date on village level for hotel',
 'Rented villa nights before booking date on village level for comfort',
 'Rented villa nights before booking date on village level for premium',
 'Rented villa nights before booking date on village level for VIP',
 'Cottage rent per person']
scale_cols2 = [
        'Physical capacity village level',
        'Physical capacity village level for hotel', 'Physical capacity village level for comfort',
        'Physical capacity village level for premium', 'Physical capacity village level for VIP', 
        'Rented villa nights before booking date on village level',
        'Rented villa nights before booking date on village level for hotel',
        'Rented villa nights before booking date on village level for comfort',
        'Rented villa nights before booking date on village level for premium',
        'Rented villa nights before booking date on village level for VIP',
        'Cottage rent per person', 'Lagged Cancellation Amount','Lagged cancellation insurance Amount','Lagged travel insurance Amount'
        
    ]
capacity_14 = horizon14[capacity_only].copy()
capacity_7 = horizon7[capacity_only].copy()
capacity_5 = horizon5[capacity_only].copy()
capacity_1 = horizon1[capacity_only].copy()


In [ ]:
#using two standard deviations for extreme prediction errors
double_standard=2*capacity_14['Occupancy'].std()
double_standard

In [ ]:
#Experiment 1
#no weather features model
def no_weather_model(split_date='2023-10-01', shapE=False,residual=False):
    
   

    frames= [capacity_14,capacity_7,capacity_5,capacity_1]
    compare = pd.DataFrame(columns=['true','predicted','Leadtime','Model','difference','negative','Date visit','extreme'])
    #model list
    models = ['XGBRegressor', 'ridge regression', 'SVR']
    #Setting the optimal parameters for each model and lead time
    optimal_params = {
        'XGBRegressor': {
            14: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}, 
            7: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200},   
            5: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200},  
            1: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}    
        },
        'ridge regression': {
            14: {'alpha': 5},  
            7: {'alpha': 5},  
            5: {'alpha': 5},   
            1: {'alpha': 5}    
        },
        'SVR': {
            14: {'C': 0.1, 'epsilon': 0.1, 'kernel': 'rbf'},  # Best Validation MAPE: 0.058
            7: {'C': 0.1, 'epsilon': 0.01, 'kernel': 'rbf'},  # Best Validation MAPE: 0.023
            5: {'C': 0.1, 'epsilon': 0.01, 'kernel': 'rbf'},  # Best Validation MAPE: 0.018
            1: {'C': 10, 'epsilon': 0.01, 'kernel': 'rbf'}    # Best Validation MAPE: 0.006
        }
    }







 
    
    naive_maape=[0.123]
    naive=[0.094]
    naive_rmse=[0.126]
    
    masecompare = pd.DataFrame(columns=['Leadtime', 'Model','MAE','MAPE','RMSE','Approach'])

    for model in models:
        
        for t in frames:
            lead=t['Leadtime'].max()
            # Perform the time-based split
            # Prepare training data
            split_date3 = '2022-09-30'
            train = t.loc[:split_date3]
            y_train = train['Occupancy']
            X_train = train.drop(columns=['Occupancy'])
            test = t.loc[split_date:]  
            print(train.shape)
            # Prepare the test data
            X_test = test.drop(columns=['Occupancy'])
            y_test = test['Occupancy']
            if model == 'XGBRegressor':
                model_instance = xgb.XGBRegressor(**optimal_params[model][lead], enable_categorical=True)
            elif model == 'ridge regression':
                model_instance = Ridge(**optimal_params[model][lead])
            elif model == 'SVR':
                model_instance = SVR(**optimal_params[model][lead])
            model_instance.fit(X_train, y_train)
            # Predict on the test set
            y_pred = model_instance.predict(X_test)
           
            compare = comparison(y_test, y_pred, lead,compare,model,X_test,double_standard)
            

            mae = np.round(mean_absolute_error(y_test, y_pred), 3)
            rmse = np.round(root_mean_squared_error(y_test, y_pred), 3)
            
            mape=np.round(mean_absolute_percentage_error(y_test, y_pred), 3)
            print(model)
            print('Test leadtime',lead)
            print(f"MAE for {model}: {mae}")
            print(f"RMSE for {model}: {rmse}")
            print(f"MAPE for {model}: {mape}")
           
            
            new_row = pd.DataFrame([{'Leadtime': lead, 'Model': model,'MAE':mae,'RMSE':rmse,'MAPE':mape,'Approach':'No_weather'}])

# Concatenate the new row with the existing DataFrame
            masecompare = pd.concat([masecompare, new_row], ignore_index=True)
            

            # Measuring feature importance
            if shapE:
                if isinstance(model_instance, xgb.XGBRegressor):
                    explainer = shap.TreeExplainer(model_instance)
                    shap_values = explainer.shap_values(X_test)
                elif isinstance(model_instance, SVR):
                    continue  # Skip SHAP for SVR
                else:
                    explainer = shap.LinearExplainer(model_instance, X_train)
                    shap_values = explainer.shap_values(X_test)
                    
                plt.figure()
                shap.summary_plot(shap_values, X_test, show=False)
                plt.title(f"SHAP Leadtime for {model} (MAPE: {mape})")
                plt.show()
            
            # Print comparison of results
       
            if residual:
                # Stationarity of errors
                time_series = compare['difference'].copy()
                plt.figure(figsize=(10, 6))
                plt.plot(time_series, label=f"Errors forecast horizon {lead} for {model} (MAE: {mae})")
                
                # Rolling mean and variance
                rolling_mean = time_series.rolling(window=lead).mean()
                rolling_std = time_series.rolling(window=lead).std()

                # Plot rolling statistics
                plt.plot(rolling_mean, color='red', label='Rolling Mean')
                plt.plot(rolling_std, color='black', label='Rolling Std')
                plt.legend()
                plt.show()
    
# Plot MAAPE comparison across different lead times and models
    plt.figure(figsize=(10, 6))
    for model_name in masecompare['Model'].unique():
        subset = masecompare[masecompare['Model'] == model_name]
        plt.plot(subset['Leadtime'], subset['MAPE'], label=model_name, marker='o')
        
    plt.axhline( naive, label='Naive Baseline', marker='x', linestyle='--', color='black')     
    plt.title('MAPE Comparison Across Lead Times and Models')
    plt.xlabel('Leadtime')
    plt.ylabel('MAPE')
    plt.legend()
    plt.show()
    # Plot MAE comparison across different lead times and models
    plt.figure(figsize=(10, 6))
    for model_name in masecompare['Model'].unique():
        subset = masecompare[masecompare['Model'] == model_name]
        plt.plot(subset['Leadtime'], subset['MAE'], label=model_name, marker='o')
        
    plt.axhline( naive, label='Naive Baseline', marker='x', linestyle='--', color='black')     
    plt.title('MAE Comparison Across Lead Times and Models')
    plt.xlabel('Leadtime')
    plt.ylabel('MAE')
    plt.legend()
    plt.show()

    return masecompare,compare
no_weather,residuals_no_weather=no_weather_model(shapE=True,residual=True)

In [ ]:
no_weather

In [ ]:
#Tuning for experiment 1 no weather features
def tuning_noweather( split_date='2023-10-01'):

    # Initialize the models
    models = [xgb.XGBRegressor(enable_categorical=True),
              Ridge(),
              SVR()]
    
    # Prepare training data
    
    frames = [capacity_16,capacity_7,capacity_5,capacity_1]
   
    
    for model in models:
        for frame in frames:
            lead=frame['Leadtime'].max()
            train,test,validation=train_test_validation(df=frame)
            y_train = train['Occupancy']
            X_train = train.drop(columns=['Occupancy'])
            
            y_val = validation['Occupancy']
            X_val = validation.drop(columns=['Occupancy'])

            if isinstance(model, xgb.XGBRegressor):
                param_grid = {
                    'n_estimators': [100,200,500],
                    'max_depth': [3, 5, 7, 14],
                    'learning_rate': [0.001, 0.01, 0.1]
                }
            elif isinstance(model, SVR):
                param_grid = {'C': [0.1, 1, 10], 'epsilon': [0.01, 0.1, 0.2],'kernel':['rbf']}
            else:
                param_grid = {'alpha': [0, 0.1, 1, 3, 5]}
        
        # Set up TimeSeriesSplit for cross-validation
            tscv = TimeSeriesSplit(n_splits=5)

            # Initialize GridSearchCV with TimeSeriesSplit
            grid_search = GridSearchCV(estimator=model, 
                                    param_grid=param_grid,
                                    scoring='neg_mean_absolute_percentage_error',  #MAAPE for scoring
                                    cv=tscv,
                                    n_jobs=-1)
            
            # Fit the grid search model
            grid_search.fit(X_train, y_train)

            # Get the best parameters and best score
            best_params = grid_search.best_params_
              
            # Initialize the model with the best parameters
            best_model = model.set_params(**best_params).fit(X_train, y_train)

            # Fit the best model 
            y_val_pred = best_model.predict(X_val)
            mape=np.round(mean_absolute_percentage_error(y_val, y_val_pred), 3)
            
            print(f"Best Parameters for Leadtime {lead} for {model.__class__.__name__}:", best_params)
            print("Best Validation MAPE Score:", mape)

tuning_noweather()

In [ ]:
#Experiment 2
#unscaled model
def unscaled_model(split_date='2023-10-01', shapE=False,residual=False):
  
   

    
    compare = pd.DataFrame(columns=['true','predicted','Leadtime','Model','difference','negative','Date visit','extreme'])
    #model list
    models = ['XGBRegressor', 'ridge regression', 'SVR']
    # Setting the optimal parameters for each model and lead time
    optimal_params = {
        'XGBRegressor': {
            1: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100},
            5: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200},
            7: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500},
            14: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200},
        },
        'Ridge': {
            1: {'alpha': 0.1},
            5: {'alpha': 1},
            7: {'alpha': 0},
            14: {'alpha': 0.1},
        },
        'SVR': {
            1: {'C': 10, 'epsilon': 0.01, 'kernel': 'rbf'},
            5: {'C': 0.1, 'epsilon': 0.01, 'kernel': 'rbf'},
            7: {'C': 0.1, 'epsilon': 0.01, 'kernel': 'rbf'},
            14: {'C': 0.1, 'epsilon': 0.1, 'kernel': 'rbf'},
        }
    }





 
    
    naive_maape=[0.123]
    naive=[0.094]
    naive_rmse=[0.126]
    
    masecompare = pd.DataFrame(columns=['Leadtime', 'Model','MAE','MAPE','RMSE','Approach'])

    for model in models:
        
        for t in frames:
            lead=t['Leadtime'].max()
            # Perform the time-based split
            # Prepare training data
            split_date3 = '2022-09-30'
            train = t.loc[:split_date3]
            y_train = train['Occupancy']
            X_train = train.drop(columns=['Occupancy'])
            test = t.loc[split_date:]  
            print(train.shape)
            # Prepare the test data
            X_test = test.drop(columns=['Occupancy'])
            y_test = test['Occupancy']
            if model == 'XGBRegressor':
                model_instance = xgb.XGBRegressor(**optimal_params[model][lead], enable_categorical=True)
            elif model == 'Ridge':
                model_instance = Ridge(**optimal_params[model][lead])
            elif model == 'SVR':
                model_instance = SVR(**optimal_params[model][lead])
            model_instance.fit(X_train, y_train)
            # Predict on the test set
            y_pred = model_instance.predict(X_test)
           
            compare = comparison(y_test, y_pred, lead,compare,model,X_test,double_standard)
            

            mae = np.round(mean_absolute_error(y_test, y_pred), 3)
            rmse = np.round(root_mean_squared_error(y_test, y_pred), 3)
            #maape_score=np.round(maape(y_test, y_pred), 3)
            mape=np.round(mean_absolute_percentage_error(y_test, y_pred), 3)
            print(model)
            print('Test leadtime',lead)
            print(f"MAE for {model}: {mae}")
            print(f"RMSE for {model}: {rmse}")
            print(f"MAPE for {model}: {mape}")
           
            
            new_row = pd.DataFrame([{'Leadtime': lead, 'Model': model,'MAE':mae,'RMSE':rmse,'MAPE':mape,'Approach':'Unscaled'}])

# Concatenate the new row with the existing DataFrame
            masecompare = pd.concat([masecompare, new_row], ignore_index=True)
            

            # Measuring feature importance
            if shapE:
                if isinstance(model_instance, xgb.XGBRegressor):
                    explainer = shap.TreeExplainer(model_instance)
                    shap_values = explainer.shap_values(X_test)
                elif isinstance(model_instance, SVR):
                    continue  # Skip SHAP for SVR
                else:
                    explainer = shap.LinearExplainer(model_instance, X_train)
                    shap_values = explainer.shap_values(X_test)
                    
                plt.figure()
                shap.summary_plot(shap_values, X_test, show=False)
                plt.title(f"SHAP Leadtime for {model} (MAPE: {mape})")
                plt.show()
            
            # Print comparison of results
       
            if residual:
                # Stationarity of errors
                time_series = compare['difference'].copy()
                plt.figure(figsize=(10, 6))
                plt.plot(time_series, label=f"Errors forecast horizon {lead} for {model} (MAE: {mae}))")
                
                # Rolling mean and variance
                rolling_mean = time_series.rolling(window=lead).mean()
                rolling_std = time_series.rolling(window=lead).std()

                # Plot rolling statistics
                plt.plot(rolling_mean, color='red', label='Rolling Mean')
                plt.plot(rolling_std, color='black', label='Rolling Std')
                plt.legend()
                plt.show()
    
# Plot MAAPE comparison across different lead times and models
    plt.figure(figsize=(10, 6))
    for model_name in masecompare['Model'].unique():
        subset = masecompare[masecompare['Model'] == model_name]
        plt.plot(subset['Leadtime'], subset['MAPE'], label=model_name, marker='o')
        
    plt.axhline( naive, label='Naive Baseline', marker='x', linestyle='--', color='black')     
    plt.title('MAPE Comparison Across Lead Times and Models')
    plt.xlabel('Leadtime')
    plt.ylabel('MAPE')
    plt.legend()
    plt.show()
    # Plot MAE comparison across different lead times and models
    plt.figure(figsize=(10, 6))
    for model_name in masecompare['Model'].unique():
        subset = masecompare[masecompare['Model'] == model_name]
        plt.plot(subset['Leadtime'], subset['MAE'], label=model_name, marker='o')
        
    plt.axhline( naive, label='Naive Baseline', marker='x', linestyle='--', color='black')     
    plt.title('MAE Comparison Across Lead Times and Models')
    plt.xlabel('Leadtime')
    plt.ylabel('MAE')
    plt.legend()
    plt.show()

    return masecompare,compare
unscaled,residuals_unscaled=unscaled_model(shapE=True,residual=True)


In [ ]:
residuals_unscaled

In [ ]:
#Tuning for experiment 2 unscaled
def tuning_unscaled( split_date='2023-10-01'):

    # Initialize the models
    models = [xgb.XGBRegressor(enable_categorical=True),
              Ridge(),
              SVR()]
    
    # Prepare training data
    
    frames = [lead16,lead7,lead5,lead1]
   
    
    for model in models:
        for frame in frames:
            lead=frame['Leadtime'].max()
            train,test,validation=train_test_validation(df=frame)
            y_train = train['Occupancy']
            X_train = train.drop(columns=['Occupancy'])
            
            y_val = validation['Occupancy']
            X_val = validation.drop(columns=['Occupancy'])

            if isinstance(model, xgb.XGBRegressor):
                param_grid = {
                    'n_estimators': [100,200,500],
                    'max_depth': [3, 5, 7, 14],
                    'learning_rate': [0.001, 0.01, 0.1]
                }
            elif isinstance(model, SVR):
                param_grid = {'C': [0.1, 1, 10], 'epsilon': [0.01, 0.1, 0.2],'kernel':['rbf']}
            else:
                param_grid = {'alpha': [0, 0.1, 1, 3, 5]}
        
        # Set up TimeSeriesSplit for cross-validation
            tscv = TimeSeriesSplit(n_splits=5)

            # Initialize GridSearchCV with TimeSeriesSplit
            grid_search = GridSearchCV(estimator=model, 
                                    param_grid=param_grid,
                                    scoring='neg_mean_absolute_percentage_error',  #MAAPE for scoring
                                    cv=tscv,
                                    n_jobs=-1)
            
            # Fit the grid search model
            grid_search.fit(X_train, y_train)

            # Get the best parameters and best score
            best_params = grid_search.best_params_
              
            # Initialize the model with the best parameters
            best_model = model.set_params(**best_params).fit(X_train, y_train)

            # Fit the best model 
            y_val_pred = best_model.predict(X_val)
            mape=np.round(mean_absolute_percentage_error(y_val, y_val_pred), 3)
            
            print(f"Best Parameters for Leadtime {lead} for {model.__class__.__name__}:", best_params)
            print("Best Validation MAPE Score:", mape)

tuning_unscaled()

In [ ]:
#Comparing models from experiment 1

def plot_best_model_approach(df1, df2,xp1=False,xp2=False,xp3=False):
    # Add an Approach column to distinguish between general and specific
    if xp1==True:
        df1['Approach'] = 'No_weather'
        df2['Approach'] = 'Unscaled'

    if xp2==True:
        df1['Approach'] = 'Scaled'
        df2['Approach'] = 'Unscaled'

    if xp3==True:
        df1['Approach'] = 'Blended'
        df2['Approach'] = 'Separate'
  


    # Concatenate the two DataFrames
    combined_df = pd.concat([df1,df2])
    leads = np.unique(combined_df['Leadtime'])
    
    # Prepare highlights DataFrames for each metric
    highlights_mae = pd.DataFrame(columns=combined_df.columns)
    highlights_MAPE = pd.DataFrame(columns=combined_df.columns)
    highlights_RMSE = pd.DataFrame(columns=combined_df.columns)
    # Identify best values for each metric
    for lead in leads:
        temp_df = combined_df[combined_df['Leadtime'] == lead]
        highlights_mae = pd.concat([highlights_mae, temp_df.sort_values(by='MAE').drop_duplicates(['Model'])], ignore_index=True)
        highlights_MAPE = pd.concat([highlights_MAPE, temp_df.sort_values(by='MAPE').drop_duplicates(['Model'])], ignore_index=True)
        highlights_RMSE = pd.concat([highlights_RMSE, temp_df.sort_values(by='RMSE').drop_duplicates(['Model'])], ignore_index=True)
    # Plot configurations
    if xp1:
        markers = {'No_weather': 's', 'Unscaled': 'o'}
    elif xp2:
        markers = {'Scaled': 's', 'Unscaled': 'o'}
    elif xp3:
        markers = {'Blended': 's', 'Separate': 'o'}
    colors = {'XGBRegressor': 'blue', 'ridge regression': 'green', 'SVR': 'red'}

    def plot_metric(metric, highlights_df, ylabel, baseline_key):
        plt.figure(figsize=(10, 8))
        for model in highlights_df['Model'].unique():
            model_data = highlights_df[highlights_df['Model'] == model]
            for approach in model_data['Approach'].unique():
                approach_data = model_data[model_data['Approach'] == approach]
                plt.scatter(
                    approach_data['Leadtime'], 
                    approach_data[metric], 
                    marker=markers[approach], 
                    color=colors[model], 
                    label=f'{model} ({approach})',
                    s=100
                )

        plt.title(f'Best Model Approaches for {metric} across forecast horizons')
        plt.xlabel('Leadtime')
        plt.ylabel(ylabel)
        plt.legend(loc='best')
        plt.show()

    # Plot the MAE and MAPE graphs
    plot_metric('MAE', highlights_mae, 'MAE', 'MAE')
    plot_metric('MAPE', highlights_MAPE, 'MAPE', 'MAPE')
    plot_metric('RMSE', highlights_RMSE, 'RMSE', 'RMSE')

    final_comparison=pd.concat([highlights_MAPE,highlights_mae,highlights_RMSE])
    final_comparison = final_comparison.drop_duplicates()
    final_comparison = final_comparison.dropna(axis=1)
    return final_comparison
# Use the function with your DataFrames
best_unscaled=plot_best_model_approach(no_weather, unscaled,True)

In [ ]:
#Experiment 2
#Robust scaling

def scale_model(split_date='2023-10-01', shapE=False,residual=False):

    compare = pd.DataFrame(columns=['true','predicted','Leadtime','Model','difference','negative','Date visit','extreme'])
    
    
    #model list
    models = ['XGBRegressor', 'ridge regression', 'SVR']
    # Define the optimal parameters for each model and lead time
    optimal_params = {
        'XGBRegressor': {
            1: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100},
            5: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200},
            7: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500},
            14: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
        },
        'ridge regression': {
            1: {'alpha': 3},
            5: {'alpha': 5},
            7: {'alpha': 5},
            14: {'alpha': 5}
        },
        'SVR': {
            1: {'C': 10, 'epsilon': 0.01, 'kernel': 'rbf'},
            5: {'C': 1, 'epsilon': 0.01, 'kernel': 'rbf'},
            7: {'C': 1, 'epsilon': 0.01, 'kernel': 'rbf'},
            14: {'C': 0.1, 'epsilon': 0.01, 'kernel': 'rbf'}
        }
    }

    
    masecompare = pd.DataFrame(columns=['Leadtime', 'Model','MAE','RMSE','MAPE','Approach'])
    scaler=RobustScaler()
    for model in models:
        
        for t in frames:
            lead=t['Leadtime'].max()
            # Perform the time-based split
            # Prepare training data
            train = t.loc[:split_date]
            y_train = train['Occupancy']
            X_train = train.drop(columns=['Occupancy'])
                #Scaling all non categorical features
            X_train[scale_cols] = scaler.fit_transform(X_train[scale_cols])
            
            test = t.loc[split_date:]  
            
            # Prepare the test data
            X_test = test.drop(columns=['Occupancy'])
            X_test[scale_cols] = scaler.transform(X_test[scale_cols])
            y_test = test['Occupancy']
            if model == 'XGBRegressor':
                model_instance = xgb.XGBRegressor(**optimal_params[model][lead], enable_categorical=True)
     
            elif model == 'ridge regression':
                model_instance = Ridge(**optimal_params[model][lead])
            elif model == 'SVR':
                model_instance = SVR(**optimal_params[model][lead])
            model_instance.fit(X_train, y_train)
            # Predict on the test set
            y_pred = model_instance.predict(X_test)
           
            compare = comparison(y_test, y_pred, lead,compare,model,X_test,double_standard)
            

            mae = np.round(mean_absolute_error(y_test, y_pred), 3)
            rmse=np.round(root_mean_squared_error(y_test, y_pred), 3)
            #maape_score=np.round(maape(y_test, y_pred), 3)
            mape=np.round(mean_absolute_percentage_error(y_test, y_pred), 3)
            print(model)
            print('Test leadtime',lead)
            print(f"MAE for {model}: {mae}")
            print(f"RMSE for {model}: {rmse}")
            print(f"MAPE for {model}: {mape}")
           
            
            new_row = pd.DataFrame([{'Leadtime': lead, 'Model': model,'MAE':mae,'MAPE':mape,'Approach':'Scaled','RMSE':rmse}])

# Concatenate the new row with the existing DataFrame
            masecompare = pd.concat([masecompare, new_row], ignore_index=True)
            

            # Measuring feature importance
            if shapE:
                if isinstance(model_instance, xgb.XGBRegressor):
                    explainer = shap.TreeExplainer(model_instance)
                    shap_values = explainer.shap_values(X_test)
                elif isinstance(model_instance, SVR):
                    continue  # Skip SHAP for SVR
                else:
                    explainer = shap.LinearExplainer(model_instance, X_train)
                    shap_values = explainer.shap_values(X_test)
                    
                plt.figure()
                shap.summary_plot(shap_values, X_test, show=False)
                plt.title(f"SHAP for {model} (MAPE: {mape})")
                plt.show()
            
            # Print comparison of results
            
            #print(compare.describe(include='all'))
            if residual:
                # Stationarity of errors
                time_series = compare['difference'].copy()
                plt.figure(figsize=(10, 6))
                plt.plot(time_series, label=f"Errors forecast horizon {lead} for {model} (MAE: {mae})")
                
                # Rolling mean and variance
                rolling_mean = time_series.rolling(window=lead).mean()
                rolling_std = time_series.rolling(window=lead).std()

                # Plot rolling statistics
                plt.plot(rolling_mean, color='red', label='Rolling Mean')
                plt.plot(rolling_std, color='black', label='Rolling Std')
                plt.legend()
                plt.show()
    
# Plot MAAPE comparison across different lead times and models
    plt.figure(figsize=(10, 6))
    for model_name in masecompare['Model'].unique():
        subset = masecompare[masecompare['Model'] == model_name]
        plt.plot(subset['Leadtime'], subset['MAPE'], label=model_name, marker='o')
        
    
    plt.title('MAPE Comparison Across Lead Times and Models')
    plt.xlabel('Leadtime')
    plt.ylabel('MAPE')
    plt.legend()
    plt.show()
    # Plot MAE comparison across different lead times and models
    plt.figure(figsize=(10, 6))
    for model_name in masecompare['Model'].unique():
        subset = masecompare[masecompare['Model'] == model_name]
        plt.plot(subset['Leadtime'], subset['MAE'], label=model_name, marker='o')
        
      
    plt.title('MAE Comparison Across Lead Times and Models')
    plt.xlabel('Leadtime')
    plt.ylabel('MAE')
    plt.legend()
    plt.show()

    return masecompare,compare
scaled_model,residuals_scale=scale_model(shapE=True,residual=True)

In [ ]:
residuals_scale


In [ ]:
#Tuning for experiment 2
#Tuning with robust scaling

def tuning_scaling( split_date='2023-10-01'):

    # Initialize the models
    models = [SVR(),xgb.XGBRegressor(enable_categorical=True),
              Ridge()]
    

    
    for model in models:
        for frame in frames:
           
            train,test,validation=train_test_validation(df=frame)
            scaler=RobustScaler()
            
            y_train = train['Occupancy']
            X_train = train.drop(columns=['Occupancy'])

            y_val=validation['Occupancy']
            X_val=validation.drop(columns=['Occupancy'])

            #Scaling all non categorical features
            X_train[scale_cols] = scaler.fit_transform(X_train[scale_cols])
            X_val[scale_cols] = scaler.transform(X_val[scale_cols])
            if isinstance(model, xgb.XGBRegressor):
                
                param_grid = {
                    'n_estimators': [100,200,500],
                    'max_depth': [3, 5, 7, 14],
                    'learning_rate': [0.001, 0.01, 0.1]
                }

              

            elif isinstance(model, SVR):
                param_grid = {'C': [0.1, 1, 10], 'epsilon': [0.01, 0.1, 0.2],'kernel':['rbf']}
            else:
                param_grid = {'alpha': [0, 0.1, 1, 3, 5]}
        
        # Set up TimeSeriesSplit for cross-validation
            tscv = TimeSeriesSplit(n_splits=5)

            # Initialize GridSearchCV with TimeSeriesSplit
            grid_search = GridSearchCV(estimator=model, 
                                    param_grid=param_grid,
                                    scoring='neg_mean_absolute_percentage_error',  #MAAPE for scoring
                                    cv=tscv,
                                    n_jobs=-1)

            # Fit the grid search model
            grid_search.fit(X_train, y_train)
            best_params = grid_search.best_params_
            best_model = grid_search.best_estimator_
            best_model = model.set_params(**best_params).fit(X_train, y_train)
# Predict on the validation set using the best model
            y_val_pred = best_model.predict(X_val)

            # Get the best parameters and best score
            
            mape=np.round(mean_absolute_percentage_error(y_val, y_val_pred), 3)
            print('Leadtime:',frame['Leadtime'].max())
            print(f"Best Parameters for {model.__class__.__name__}:", best_params)
            print("Best Validation MAPE Score:", mape)
    
tuning_scaling()

In [ ]:
#Comparing scaling with unscaled best models
print('Scaled models')
print(scaled_model)
print('Unscaled best')
print(best_unscaled.sort_values(by='Model'))
plot_best_model_approach(scaled_model,best_unscaled ,xp2=True)

In [ ]:
#Experiment 3
#different model for different leads, different pca for weather and capacity
def separate_pca(split_date='2023-10-01', shapE=False, residual=False):
    # Define the test DataFrames based on lead times
    
    compare = pd.DataFrame(columns=['true','predicted','Leadtime','Model','difference','negative','Date visit','extreme'])
    w_components=2
    c_components=2
    scaler=RobustScaler()


   
    # model list
    models = ['XGBRegressor', 'ridge regression', 'SVR']
    # Define the optimal parameters for each model and lead time
    optimal_params = {
        'XGBRegressor': {
            14: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500},
            7: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500},
            5: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500},
            1: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}
        },
        'ridge regression': {
            14: {'alpha': 5},
            7: {'alpha': 5},
            5: {'alpha': 5},
            1: {'alpha': 5}
        },
        'SVR': {
            14: {'C': 0.1, 'epsilon': 0.1, 'kernel': 'rbf'},
            7: {'C': 1, 'epsilon': 0.2, 'kernel': 'rbf'},
            5: {'C': 1, 'epsilon': 0.2, 'kernel': 'rbf'},
            1: {'C': 1, 'epsilon': 0.01, 'kernel': 'rbf'}
        }
    }


    
    masecompare = pd.DataFrame(columns=['Leadtime', 'Model','MAE','MAPE','RMSE','Approach'])

    for model in models:
        
        for t in frames:
            lead=t['Leadtime'].max()
            # Perform the time-based split
            # Prepare training data
            split_date3 = '2022-09-30'
            train = t.loc[:split_date3]
            y_train = train['Occupancy']
            X_train = train.drop(columns=['Occupancy'])
            test = t.loc[split_date:]  

            # Prepare the test data
            X_test = test.drop(columns=['Occupancy'])
            y_test = test['Occupancy']
            #Copy for comparison date index since PCA drops index
            a=X_test.copy()

                            #Scaling all non categorical features
            X_train[scale_cols] = scaler.fit_transform(X_train[scale_cols])
        
            # Apply PCA for the training set
            pca_weather = PCA(n_components=w_components)
            pca_capacity = PCA(n_components=c_components)
            
            X_train_weather = pd.DataFrame(pca_weather.fit_transform(X_train[weather_cols]), columns=[f'PC_weather_{i+1}' for i in range(w_components)])
            X_train_capacity = pd.DataFrame(pca_capacity.fit_transform(X_train[capacity_cols]), columns=[f'PC_capacity_{i+1}' for i in range(c_components)])
            X_train = pd.concat([X_train_weather.reset_index(drop=True), X_train_capacity.reset_index(drop=True), train[cat_cols].reset_index(drop=True)], axis=1)

                            #Scaling all non categorical features
            X_test[scale_cols] = scaler.transform(X_test[scale_cols])
        
            # Apply PCA for the test set
          
            X_test_weather = pd.DataFrame(pca_weather.transform(X_test[weather_cols]), columns=[f'PC_weather_{i+1}' for i in range(w_components)])
            X_test_capacity = pd.DataFrame(pca_capacity.transform(X_test[capacity_cols]), columns=[f'PC_capacity_{i+1}' for i in range(c_components)])
            X_test = pd.concat([X_test_weather.reset_index(drop=True), X_test_capacity.reset_index(drop=True), test[cat_cols].reset_index(drop=True)], axis=1)
            

            if model == 'XGBRegressor':
                model_instance = xgb.XGBRegressor(**optimal_params[model][lead], enable_categorical=True)
            elif model == 'ridge regression':
                model_instance = Ridge(**optimal_params[model][lead])
            elif model == 'SVR':
                model_instance = SVR(**optimal_params[model][lead])
            model_instance.fit(X_train, y_train)
            # Predict on the test set
            y_pred = model_instance.predict(X_test)
           
            compare = comparison(y_test, y_pred, lead,compare,model,a,double_standard)
            

            mae = np.round(mean_absolute_error(y_test, y_pred), 3)
            mape=np.round(mean_absolute_percentage_error(y_test, y_pred), 3)
            rmse=np.round(root_mean_squared_error(y_test, y_pred), 3)
            print(model)
            print('Test leadtime',lead)
            print(f"MAE for {model}: {mae}")
            print(f"MAPE for {model}: {mape}")
           
            
            new_row = pd.DataFrame([{'Leadtime': lead, 'Model': model,'MAE':mae,'MAPE':mape,'RMSE':rmse,'Approach':'Separate'}])

# Concatenate the new row with the existing DataFrame
            masecompare = pd.concat([masecompare, new_row], ignore_index=True)
            

            # Measuring feature importance
            if shapE:
                if model=='XGBRegressor':
                    explainer = shap.TreeExplainer(model_instance)
                    shap_values = explainer.shap_values(X_test)
                elif model=='SVR':
                    continue  # Skip SHAP for SVR
                else:
                    explainer = shap.LinearExplainer(model_instance, X_train)
                    shap_values = explainer.shap_values(X_test)
                    
                plt.figure()
                shap.summary_plot(shap_values, X_test, show=False)
                plt.title(f"SHAP Leadtime {lead} for {model} (MAPE: {mape})")
                plt.show()
            
            # Print comparison of results
           
            #print(compare.describe(include='all'))
            if residual:
                # Stationarity of errors
                time_series = compare['difference'].copy()
                plt.figure(figsize=(10, 6))
                plt.plot(time_series, label=f"Errors forecast horizon {lead} for {model} (MAE: {mae})")
                
                # Rolling mean and variance
                rolling_mean = time_series.rolling(window=lead).mean()
                rolling_std = time_series.rolling(window=lead).std()

                # Plot rolling statistics
                plt.plot(rolling_mean, color='red', label='Rolling Mean')
                plt.plot(rolling_std, color='black', label='Rolling Std')
                plt.legend()
                plt.show()
    
# Plot MAPE comparison across different lead times and models
    plt.figure(figsize=(10, 6))
    for model_name in masecompare['Model'].unique():
        subset = masecompare[masecompare['Model'] == model_name]
        plt.plot(subset['Leadtime'], subset['MAPE'], label=model_name, marker='o')
        
    #plt.plot(np.unique(masecompare['Leadtime']), naive_MAPE, label='Naive Baseline', marker='x', linestyle='--', color='black')   
    plt.title('MAPE Comparison Across Lead Times and Models')
    plt.xlabel('Leadtime')
    plt.ylabel('MAPE')
    plt.legend()
    plt.show()
    # Plot MAE comparison across different lead times and models
    plt.figure(figsize=(10, 6))
    for model_name in masecompare['Model'].unique():
        subset = masecompare[masecompare['Model'] == model_name]
        plt.plot(subset['Leadtime'], subset['MAE'], label=model_name, marker='o')
        
    #plt.plot(np.unique(masecompare['Leadtime']), sorted(naive), label='Naive Baseline', marker='x', linestyle='--', color='black')   
    plt.title('MAE Comparison Across Lead Times and Models')
    plt.xlabel('Leadtime')
    plt.ylabel('MAE')
    plt.legend()
    plt.show()

    return masecompare,compare
# Call the function
separate,residuals_separate=separate_pca(shapE=True,residual=True)

In [ ]:
residuals_separate

In [ ]:
def tuning_separate(df, w_components=2,c_components=2):
    # Define columns to scale
    split_date='2023-10-01'

   

    # Initialize models
    models = [ SVR(), Ridge(),xgb.XGBRegressor(enable_categorical=True)]
    scaler = RobustScaler()
    # Set up TimeSeriesSplit for cross-validation
    tscv = TimeSeriesSplit(n_splits=5)
    best_params = {model.__class__.__name__: {} for model in models}
    for frame in frames:
        train,test,validation=train_test_validation(df=frame)
        y_train = train['Occupancy']
        X_train = train.drop(columns=['Occupancy'])

        y_val=validation['Occupancy']
        X_val=validation.drop(columns=['Occupancy'])

            #Scaling all non categorical features
        X_train[scale_cols] = scaler.fit_transform(X_train[scale_cols])
        X_val[scale_cols] = scaler.transform(X_val[scale_cols])

        # Apply PCA for the training set
        pca_weather = PCA(n_components=w_components)
        pca_capacity = PCA(n_components=c_components)
        X_train_weather = pd.DataFrame(pca_weather.fit_transform(X_train[weather_cols]), columns=[f'PC_weather_{i+1}' for i in range(w_components)])
        X_train_capacity = pd.DataFrame(pca_capacity.fit_transform(X_train[capacity_cols]), columns=[f'PC_capacity_{i+1}' for i in range(c_components)])
        X_train = pd.concat([X_train_weather.reset_index(drop=True), X_train_capacity.reset_index(drop=True), X_train[cat_cols].reset_index(drop=True)], axis=1)

        # Apply PCA for the validation set 
        X_val_weather = pd.DataFrame(pca_weather.transform(X_val[weather_cols]), columns=[f'PC_weather_{i+1}' for i in range(w_components)])
        X_val_capacity = pd.DataFrame(pca_capacity.transform(X_val[capacity_cols]), columns=[f'PC_capacity_{i+1}' for i in range(c_components)])
        X_val = pd.concat([X_val_weather.reset_index(drop=True), X_val_capacity.reset_index(drop=True), X_val[cat_cols].reset_index(drop=True)], axis=1)

        for model in models:
            print(f"\nTuning Model: {model.__class__.__name__}")
            
            # Parameter grid
            if isinstance(model, xgb.XGBRegressor):
                param_grid = {
                    'n_estimators': [100, 200, 500],
                    'max_depth': [3, 5, 7, 14],
                    'learning_rate': [0.001, 0.01, 0.1]
                }
            elif isinstance(model, SVR):
                param_grid = {
                    'C': [0.1, 1, 10],
                    'epsilon': [0.01, 0.1, 0.2],
                    'kernel': ['rbf']
                }
            else:
                param_grid = {'alpha': [0, 0.1, 1, 3,5]}

            # Set up TimeSeriesSplit for cross-validation
            tscv = TimeSeriesSplit(n_splits=5)

            # Initialize GridSearchCV with TimeSeriesSplit
            grid_search = GridSearchCV(estimator=model, 
                                    param_grid=param_grid,
                                    scoring='neg_mean_absolute_percentage_error',  #MAAPE for scoring
                                    cv=tscv,
                                    n_jobs=-1)
            
            # Fit the grid search model
            grid_search.fit(X_train, y_train)

            # Get the best parameters and best score
            best_params = grid_search.best_params_
              
            # Initialize the model with the best parameters
            best_model = model.set_params(**best_params).fit(X_train, y_train)

            # Fit the best model 
            y_val_pred = best_model.predict(X_val)
            mape=np.round(mean_absolute_percentage_error(y_val, y_val_pred), 3)
            
            print(f"Best Parameters for Leadtime {frame['Leadtime'].max()} for {model.__class__.__name__}:", best_params)
            print("Best Validation MAPE Score:", mape)
tuning_separate(df)

In [ ]:
#Experiment 3
#different model for different leads, blended pca weather and capacity
def blended_pca(split_date='2023-10-01', shapE=False, residual=False):

    compare = pd.DataFrame(columns=['true','predicted','Leadtime','Model','difference','negative','Date visit','extreme'])

# Define the optimal parameters for each model and lead time
  
    optimal_params = {
        'XGBRegressor': {
            1: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100},
            5: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100},
            7: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500},
            14: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200}
        },
        'ridge regression': {
            1: {'alpha': 5},
            5: {'alpha': 3},
            7: {'alpha': 3},
            14: {'alpha': 5}
        },
        'SVR': {
            1: {'C': 1, 'epsilon': 0.1, 'kernel': 'rbf'},
            5: {'C': 0.1, 'epsilon': 0.1, 'kernel': 'rbf'},
            7: {'C': 0.1, 'epsilon': 0.1, 'kernel': 'rbf'},
            14: {'C': 0.1, 'epsilon': 0.1, 'kernel': 'rbf'}
        }
    }


    # setting up a dataframe to store performance metrics
    masecompare = pd.DataFrame(columns=['Leadtime', 'Model', 'MAE', 'MAPE',"RMSE",'Approach'])

    # Iterate through the models and lead times
    for model in ['XGBRegressor', 'ridge regression', 'SVR']:
        for frame in frames:
            lead_time = frame['Leadtime'].max()
            lead=lead_time
            # Assign model with optimal parameters for the given lead time
            if model == 'XGBRegressor':
                model_instance = xgb.XGBRegressor( **optimal_params[model][lead_time],enable_categorical=True)
            elif model == 'ridge regression':
                model_instance = Ridge(**optimal_params[model][lead_time])
            elif model == 'SVR':
                model_instance = SVR(**optimal_params[model][lead_time])
           

             
            split_date3 = '2022-09-30'
            train = frame.loc[:split_date3]
            test = frame.loc[split_date:]  
            y_train = train['Occupancy']
            X_train = train.drop(columns=['Occupancy'])
            # Prepare the test data
            X_test = test.drop(columns=['Occupancy'])
            a=X_test.copy()
            y_test = test['Occupancy']
                # Scale data separately for train and test sets
            scaler = RobustScaler()
            X_train[scale_cols] = scaler.fit_transform(X_train[scale_cols])
            X_test[scale_cols] = scaler.transform(test[scale_cols])

         #Apply pca to train and test
            pca_total = PCA(n_components=2)
            X_train_pca = pd.DataFrame(pca_total.fit_transform(X_train[scale_cols]), columns=[f'PC_{i+1}' for i in range(2)])
            X_train = pd.concat([X_train_pca.reset_index(drop=True), X_train[cat_cols].reset_index(drop=True)], axis=1)
            
            X_test_pca=pd.DataFrame(pca_total.transform(X_test[scale_cols]), columns=[f'PC_{i+1}' for i in range(2)])
            X_test = pd.concat([X_test_pca.reset_index(drop=True), X_test[cat_cols].reset_index(drop=True)], axis=1)
    
            # Train the model
            model_instance.fit(X_train, y_train)

            # Predict on the test set
            y_pred = model_instance.predict(X_test)

            # Evaluate metrics
            mae = np.round(mean_absolute_error(y_test, y_pred), 3)
            rmse= np.round(root_mean_squared_error(y_test, y_pred), 3)
            mape = np.round(mean_absolute_percentage_error(y_test, y_pred), 3)

            # Store results
            new_row = pd.DataFrame([{'Leadtime': lead_time, 'Model': model, 'MAE': mae, 'MAPE': mape,'RMSE':rmse,'Approach':'Blended'}])
            masecompare = pd.concat([masecompare, new_row], ignore_index=True)
            compare = comparison(y_test, y_pred, lead_time,compare,model,a,double_standard)
            # Print evaluation results
            print(f"Lead time: {lead_time}, Model: {model}")
            print(f"MAE: {mae}, MAPE: {mape}")
            if shapE==True:
          
                if isinstance(model_instance, xgb.XGBRegressor):
                    explainer = shap.TreeExplainer(model_instance)
                    shap_values = explainer.shap_values(X_test)
                elif isinstance(model_instance, SVR):
                    continue  # Skip SHAP for SVR
             
                elif isinstance(model_instance, Ridge):
                    explainer = shap.LinearExplainer(model_instance, X_train)
                    shap_values = explainer.shap_values(X_test)
      
                    
                plt.figure(figsize=(10, 6))
                shap.summary_plot(shap_values, X_test, show=False)
                plt.title(f"SHAP Leadtime {lead_time} for {model} (MAPE: {mape})")
                plt.show()
            if residual:
                # Stationarity of errors
                time_series = compare['difference'].copy()
                plt.figure(figsize=(10, 6))
                plt.plot(time_series, label=f"Errors forecast horizon {lead} for {model} (MAE: {mae})")
                
                # Rolling mean and variance
                rolling_mean = time_series.rolling(window=lead).mean()
                rolling_std = time_series.rolling(window=lead).std()

                # Plot rolling statistics
                plt.plot(rolling_mean, color='red', label='Rolling Mean')
                plt.plot(rolling_std, color='black', label='Rolling Std')
                plt.legend()
                plt.show()
        print("")
    # Plot comparison of MAAPE across different lead times and models
    plt.figure(figsize=(10, 6))
    for model_name in masecompare['Model'].unique():
        subset = masecompare[masecompare['Model'] == model_name]
        plt.plot(subset['Leadtime'], subset['MAPE'], label=model_name, marker='o')
    
 
    plt.title('MAPE Comparison Across Lead Times and Models')
    plt.xlabel('Leadtime')
    plt.ylabel('MAAPE')
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 6))
    for model_name in masecompare['Model'].unique():
        subset = masecompare[masecompare['Model'] == model_name]
        plt.plot(subset['Leadtime'], subset['MAE'], label=model_name, marker='o')
    
   
    plt.title('MAE Comparison Across Lead Times and Models')
    plt.xlabel('Leadtime')
    plt.ylabel('MAE')
    plt.legend()
    plt.show()
    return masecompare,compare
# Call the function
blended,residuals_blended=blended_pca(shapE=True,residual=True)

In [ ]:
residuals_blended

In [ ]:
#Tuning for experiment 3
# Tuning with pca in each fold and pca separately for weather and capacity
def tuning_blended(df, n_components=2):
   

    # Initialize models
    models = [ SVR(), Ridge(),xgb.XGBRegressor(enable_categorical=True)]
    scaler = RobustScaler()
    # Set up TimeSeriesSplit for cross-validation
    tscv = TimeSeriesSplit(n_splits=5)
    best_params = {model.__class__.__name__: {} for model in models}
    for frame in frames:
        train,test,validation=train_test_validation(df=frame)
        y_train = train['Occupancy']
        X_train = train.drop(columns=['Occupancy'])

        y_val=validation['Occupancy']
        X_val=validation.drop(columns=['Occupancy'])
            #Scaling all non categorical features
        X_train[scale_cols] = scaler.fit_transform(X_train[scale_cols])
        X_val[scale_cols] = scaler.transform(X_val[scale_cols])
        #Apply pca
        pca_total = PCA(n_components=n_components)
        X_train_pca = pd.DataFrame(pca_total.fit_transform(X_train[scale_cols]), columns=[f'PC_{i+1}' for i in range(n_components)])
        X_train = pd.concat([X_train_pca.reset_index(drop=True), X_train[cat_cols].reset_index(drop=True)], axis=1)
        
        X_val_pca = pd.DataFrame(pca_total.transform(X_val[scale_cols]), columns=[f'PC_{i+1}' for i in range(n_components)])
        X_val = pd.concat([X_val_pca.reset_index(drop=True), X_val[cat_cols].reset_index(drop=True)], axis=1)


        for model in models:
            print(f"\nTuning Model: {model.__class__.__name__}")
            
            # Parameter grid
            if isinstance(model, xgb.XGBRegressor):
                param_grid = {'n_estimators': [100, 200, 500], 'max_depth': [3, 5, 7,14], 'learning_rate': [0.001, 0.01, 0.1]}
            elif isinstance(model, SVR):
                param_grid = {'C': [0.1, 1, 10], 'epsilon': [0.01, 0.1, 0.2], 'kernel': ['rbf']}
            else:
                param_grid = {'alpha': [0, 0.1, 1, 3, 5]}

            # Set up TimeSeriesSplit for cross-validation
            tscv = TimeSeriesSplit(n_splits=5)

            # Initialize GridSearchCV with TimeSeriesSplit
            grid_search = GridSearchCV(estimator=model, 
                                    param_grid=param_grid,
                                    scoring='neg_mean_absolute_percentage_error',  #MAAPE for scoring
                                    cv=tscv,
                                    n_jobs=-1)
            
            # Fit the grid search model
            grid_search.fit(X_train, y_train)

            # Get the best parameters and best score
            best_params = grid_search.best_params_
                
            # Initialize the model with the best parameters
            best_model = model.set_params(**best_params).fit(X_train, y_train)

            # Fit the best model 
            y_val_pred = best_model.predict(X_val)
            mape=np.round(mean_absolute_percentage_error(y_val, y_val_pred), 3)
            
            print(f"Best Parameters for Leadtime {frame['Leadtime'].max()} for {model.__class__.__name__}:", best_params)
            print("Best Validation MAPE Score:", mape)

tuning_blended(df)

In [ ]:
#Comparing scaling with unscaled best models
print('Separate PCA')
print(separate)
print('Blended PCA')
print(blended)
print('Common PCA best')
best_pca=plot_best_model_approach(blended,separate ,xp3=True)
print(best_pca.sort_values(by='Model'))


In [ ]:
#Comparing all models and trainig approaches
all_df=pd.concat([unscaled,separate,scaled_model,blended,no_weather],ignore_index=True)
def best_model(all_df=all_df):
    xgb_df = all_df[all_df['Model'] == 'XGBRegressor']
    svr_df = all_df[all_df['Model'] == 'SVR']
    ridge_df = all_df[all_df['Model'] == 'Ridge']

    # XGBRegressor
    xgb_df = xgb_df.sort_values(by='MAE')
    xgb_best_mae = xgb_df.drop_duplicates(subset=['Leadtime'])
    print('Best MAE')
    print(xgb_best_mae)
    
    xgb_df = xgb_df.sort_values(by='RMSE')
    xgb_best_rmse = xgb_df.drop_duplicates(subset=['Leadtime'])
    print('Best RMSE')
    print(xgb_best_rmse)
    
    xgb_df = xgb_df.sort_values(by='MAPE')
    xgb_best_maape = xgb_df.drop_duplicates(subset=['Leadtime'])
    print('Best MAPE')
    print(xgb_best_maape)
    print('')
    
    # SVR
    svr_df = svr_df.sort_values(by='MAE')
    svr_best_mae = svr_df.drop_duplicates(subset=['Leadtime'])
    print('Best MAE')
    print(svr_best_mae)
    
    svr_df = svr_df.sort_values(by='RMSE')
    svr_best_rmse = svr_df.drop_duplicates(subset=['Leadtime'])
    print('Best RMSE')
    print(svr_best_rmse)
    
    svr_df = svr_df.sort_values(by='MAPE')
    svr_best_maape = svr_df.drop_duplicates(subset=['Leadtime'])
    print('Best MAPE')
    print(svr_best_maape)
    print('')
    
    # Ridge
    ridge_df = ridge_df.sort_values(by='MAE')
    ridge_best_mae = ridge_df.drop_duplicates(subset=['Leadtime'])
    print('Best MAE')
    print(ridge_best_mae)
    
    ridge_df = ridge_df.sort_values(by='RMSE')
    ridge_best_rmse = ridge_df.drop_duplicates(subset=['Leadtime'])
    print('Best RMSE')
    print(ridge_best_rmse)
    
    ridge_df = ridge_df.sort_values(by='MAPE')
    ridge_best_maape = ridge_df.drop_duplicates(subset=['Leadtime'])
    print('Best MAPE')
    print(ridge_best_maape)
    print('')
best_model()


In [ ]:
all_df=pd.concat([separate,blended],ignore_index=True)
best_model(all_df)

In [ ]:
#Comparing all models and trainig approaches except the no_weather one
all_df=pd.concat([unscaled,separate,scaled_model,blended],ignore_index=True)
best_model(all_df)

In [ ]:
blended

In [ ]:
def performance_plots(df):


    all_xgb = df[df['Model'] == 'XGBRegressor']
    all_ridge = df[df['Model'] == 'ridge regression']
    all_svr = df[df['Model'] == 'SVR']
    frames = [all_xgb, all_ridge, all_svr]
    
    # Define colors for each lead time
    lead_colors = {1: 'blue', 5: 'orange', 7: 'green', 14: 'purple'}
    
    # Mapping approaches to numeric positions
    approaches = ['Unscaled', 'Separate', 'Scaled', 'Blended']
    approach_positions = {approach: i for i, approach in enumerate(approaches)}
    
    # Threshold for small differences
    threshold = 0.003

    for frame in frames:
        plt.figure(figsize=(10, 8))
        
        # Use a set to track which lead times have been added to the legend
        used_leads = set()
        
        for approach in approaches:
            temp_df = frame[frame['Approach'] == approach]
            if temp_df.empty:
                continue
            
            x_position = approach_positions[approach]
            
            # Sort the DataFrame by MAE for this approach
            temp_df = temp_df.sort_values(by='MAE').reset_index(drop=True)
            
            # Alternate offsets for points based on the threshold
            last_y = None
            offset_sign = -6  
            
            for i, (y, lead) in enumerate(zip(temp_df['MAE'], temp_df['Leadtime'])):
                if last_y is not None and abs(y - last_y) < threshold:
                    offset = offset_sign * 0.05
                    align = 'right' if offset_sign == -1 else 'left'
                    offset_sign *= -.5  # Alternate sign
                else:
                    offset = 0.2
                    align = 'center'
                
                last_y = y
                
                # Only add a label for the legend if the lead time hasn't been added yet
                label = f'Leadtime {lead}' if lead not in used_leads else ""
                if label:
                    used_leads.add(lead)
                
                plt.scatter(x_position, y, color=lead_colors[lead], label=label)
                plt.text(x_position + offset, y, f'{y:.3f}', color=lead_colors[lead], ha=align, va='center', fontsize=8)
        
        # Adjust x-axis ticks and labels
        plt.xticks(range(len(approaches)), approaches)
        plt.title(f"MAE for {frame['Model'].iloc[0]}")
        plt.xlabel("Approach")
        plt.ylabel("MAE")
        plt.margins(x=0.2, y=0.2)
        # Position legend outside the plot to the right
        plt.legend(title="Forecast horizon", bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()  # Adjust layout to make room for the legend
        plt.show()

    for frame in frames:
        plt.figure(figsize=(10, 8))
        
        # Use a set to track which lead times have been added to the legend
        used_leads = set()
        
        for approach in approaches:
            temp_df = frame[frame['Approach'] == approach]
            if temp_df.empty:
                continue
            
            x_position = approach_positions[approach]
            
            # Sort the DataFrame by MAE for this approach
            temp_df = temp_df.sort_values(by='RMSE').reset_index(drop=True)
            
            # Alternate offsets for points based on the threshold
            last_y = None
            offset_sign = -6  # Start with -1 for alternating logic
            
            for i, (y, lead) in enumerate(zip(temp_df['RMSE'], temp_df['Leadtime'])):
                if last_y is not None and abs(y - last_y) < threshold:
                    offset = offset_sign * 0.05
                    align = 'right' if offset_sign == -1 else 'left'
                    offset_sign *= -.5  # Alternate sign
                else:
                    offset = 0.2
                    align = 'center'
                
                last_y = y
                
                # Only add a label for the legend if the lead time hasn't been added yet
                label = f'Leadtime {lead}' if lead not in used_leads else ""
                if label:
                    used_leads.add(lead)
                
                plt.scatter(x_position, y, color=lead_colors[lead], label=label)
                plt.text(x_position + offset, y, f'{y:.3f}', color=lead_colors[lead], ha=align, va='center', fontsize=8)
        
        # Adjust x-axis ticks and labels
        plt.xticks(range(len(approaches)), approaches)
        plt.title(f"RMSE for {frame['Model'].iloc[0]}")
        plt.xlabel("Approach")
        plt.ylabel("RMSE")
        plt.margins(x=0.2, y=0.2)
        # Position legend outside the plot to the right
        plt.legend(title="Forecast horizon", bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()  # Adjust layout to make room for the legend
        plt.show()

    for frame in frames:
        plt.figure(figsize=(10, 8))
        
        # Use a set to track which lead times have been added to the legend
        used_leads = set()
        
        for approach in approaches:
            temp_df = frame[frame['Approach'] == approach]
            if temp_df.empty:
                continue

            x_position = approach_positions[approach]

            # Sort the DataFrame by MAPE for this approach
            temp_df = temp_df.sort_values(by='MAPE').reset_index(drop=True)

            # Alternate offsets for points based on the threshold
            last_y = None
            offset_sign = -6  # Start with -1 for alternating logic
            height_offset = 0
            for i, (y, lead) in enumerate(zip(temp_df['MAPE'], temp_df['Leadtime'])):
                # Check if current and previous y-values are close enough
                if last_y is not None and abs(y - last_y) < threshold:
                    offset = offset_sign * 0.05
                    align = 'right' if offset_sign == -1 else 'left'
                    offset_sign *= -0.5  # Alternate sign
                else:
                    offset = 0.2
                    align = 'center'
                
                last_y = y

                # Check if model is SVR, lead is 5, and the approach is Blended
                if 'SVR' in frame['Model'].values and lead == 5 and 'Blended' in frame['Approach'].values:
                    height_offset = 0.002  # Apply height offset only for this specific condition

                # Only add a label for the legend if the lead time hasn't been added yet
                label = f'Leadtime {lead}' if lead not in used_leads else ""
                if label:
                    used_leads.add(lead)

                plt.scatter(x_position, y, color=lead_colors[lead], label=label)
                plt.text(x_position + offset, y + height_offset, f'{y:.3f}', color=lead_colors[lead], ha=align, va='center', fontsize=8)

        # Adjust x-axis ticks and labels
        plt.xticks(range(len(approaches)), approaches)
        plt.title(f"MAPE for {frame['Model'].iloc[0]}")
        plt.xlabel("Approach")
        plt.ylabel("MAPE")
        plt.margins(x=0.2, y=0.2)
        # Position legend outside the plot to the right
        plt.legend(title="Forecast horizon", bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()  # Adjust layout to make room for the legend
        plt.show()



performance_plots(all_df)

In [ ]:
np.unique(residuals_scale[residuals_scale['extreme']==True]['Date visit'])



In [ ]:
residuals_scale[residuals_scale['extreme']==True].head(-1)

In [ ]:
print(np.unique(residuals_separate[residuals_separate['extreme']==True]['Date visit']))
print(residuals_separate[residuals_separate['extreme']==True]['difference'].mean())
residuals_separate[residuals_separate['extreme']==True].head(-1)


In [ ]:
print(np.unique(residuals_blended[residuals_blended['extreme']==True]['Date visit']))
print(residuals_blended[residuals_blended['extreme']==True]['difference'].mean())
residuals_blended[residuals_blended['extreme']==True].head(-1)


In [43]:
def error_analysis(df,extensive=False):

     # Define colors for each lead time
     #Can be made different, but they will just overcolor the previous one
     #opacity can be set at varying rates but it makes interpretation more difficult
     #I keep the lead time distinction to have the percentage for each horizon included in the plot
    lead_colors = {1: 'orange', 5: 'orange', 7: 'orange', 14: 'orange'}
    alphas={1:0.9,5:0.7,7:0.5,14:0.3}
    for algo in np.unique(df['Model']):
        filter = df[df['Model'] == algo]
        std_dev = filter.groupby('Leadtime')['difference'].std()
        median=filter.groupby('Leadtime')['difference'].median()
        print('Median')
        print(filter.groupby('Leadtime')['difference'].median())
        print(filter.groupby('Leadtime')['difference'].mean())
        impossible = filter['extreme'].sum()/filter.shape[0]*100
        #Plot the differences for the selected model and lead time
        plt.figure(figsize=(10, 6))
        plt.scatter(filter['Leadtime'], filter['difference'], marker='o', color='b')
        #plt.plot(std_dev.index, std_dev.values, marker='x', linestyle='--', color='r', label='Standard Deviation')
        plt.scatter(median.index,median.values,label="Median",color='r')
        plt.xlabel('Forecast horizon')
        plt.ylabel('Difference (Truth - Predictions)')
        plt.title(f'Residuals for {algo} with {np.round(impossible,2)}% of predictions above 2 standard deviations')
        plt.show()
        
        if extensive == True:
            plt.figure(figsize=(10, 6))
            temp_df=filter[filter['Leadtime']==1]
            plt.plot(temp_df['Date visit'],temp_df['true'],label='True occupancy',color='b')
            
            plt.title(f'True occupancy and extreme {algo} residuals')
            
            model_extreme=filter[filter['extreme']==True]
            for lead in np.unique(model_extreme['Leadtime']):
                #calculates extreme value percentage for each forecast horizon
                impossible = np.round(model_extreme[model_extreme['Leadtime']==lead]['extreme'].sum()/filter.shape[0]*100,2)
                #Drops duplicates for same day, same horizon to declutter plotting
                model_extreme2=model_extreme.drop_duplicates(subset=['Leadtime','Date visit'])
                
                plt.scatter(model_extreme2['Date visit'],model_extreme2['predicted'],color=lead_colors[lead], label=f'Forecast horizon {lead} with {impossible}% extreme residuals',marker='.')
                plt.legend()
            plt.show()
   




In [ ]:
error_analysis(residuals_naive,True)


In [ ]:
error_analysis(residuals_no_weather,True)

In [ ]:
error_analysis(residuals_unscaled,True)


In [ ]:
error_analysis(residuals_scale,True)

In [ ]:
error_analysis(residuals_separate,True)

In [ ]:
error_analysis(residuals_blended,True)